In [2]:
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
df = pd.read_csv(rf'/Users/danielchung/Downloads/TIP PARAM - Sheet1.csv')

In [7]:
df

,PRICE_INDEX,ROUTE,AP_INV,INVENTORY_CARRIER_CD,P_OPTION,P_PCT,PRICE_INDEX_NUM
0,I0,LHRDEN,180,BA,P1,0.21,0
1,I0,LHRDEN,180,BA,P2,0.32,0
2,I0,LHRDEN,180,AA,P1,0.21,0
3,I0,LHRDEN,180,AA,P2,0.32,0
4,I0,LHRDEN,120,BA,P1,9.00,0
...,...,...,...,...,...,...,...
211,I9,LHRDEN,120,AA,P2,0.09,9
212,I9,LHRDEN,100,BA,P1,0.20,9
213,I9,LHRDEN,100,BA,P2,0.30,9
214,I9,LHRDEN,100,AA,P1,0.20,9


In [62]:
pivot_df = df.pivot_table(
    index=["PRICE_INDEX", "PRICE_INDEX_NUM", "ROUTE", "INVENTORY_CARRIER_CD"],
    columns=["AP_INV", "P_OPTION"],
    values="P_PCT",
    aggfunc='max'
)

# Sort the 'AP_INV' level of the columns in descending order while keeping 'P_OPTION' in ascending order
# Create a custom sort function for this
def custom_sort(df, level_1_desc, level_2_asc):
    # Get the MultiIndex levels
    level_1, level_2 = df.columns.levels
    # Sort the first level in descending order
    level_1_sorted = sorted(level_1, reverse=True)
    # Sort the second level in ascending order
    level_2_sorted = sorted(level_2)
    # Create a new MultiIndex with the product of the sorted levels
    new_index = pd.MultiIndex.from_product([level_1_sorted, level_2_sorted], names=df.columns.names)
    # Reindex the DataFrame with the new MultiIndex
    df = df.reindex(columns=new_index)
    return df

# Apply the custom sorting function
pivot_df = custom_sort(pivot_df, 'AP_INV', 'P_OPTION')

# Flatten the multi-level column index by joining the levels with an underscore
# This also trims any excess padding that might cause additional underscores
pivot_df.columns = ['_'.join(str(level).strip() for level in col).rstrip('_') for col in pivot_df.columns.values]


# Reset the index to make the DataFrame flat
flat_df = pivot_df.reset_index()

In [64]:

# Flatten the multi-level column index by joining the levels with an underscore
# This also trims any excess padding that might cause additional underscores
pivot_df.columns = ['_'.join(str(level).strip() for level in col).rstrip('_') for col in pivot_df.columns.values]


# Reset the index to make the DataFrame flat
flat_df = pivot_df.reset_index()


In [66]:
flat_df.dtypes

PRICE_INDEX              object
PRICE_INDEX_NUM           int64
ROUTE                    object
INVENTORY_CARRIER_CD     object
180_P1                  float64
180_P2                  float64
120_P1                  float64
120_P2                  float64
100_P1                  float64
100_P2                  float64
dtype: object

In [24]:
flat_df

,PRICE_INDEX,PRICE_INDEX_NUM,ROUTE,INVENTORY_CARRIER_CD,1_8_0___P_2,1_8_0___P_1,1_2_0___P_2,1_2_0___P_1,1_0_0___P_2,1_0_0___P_1
0,I0,0,LHRDEN,AA,0.32,0.21,9.00,9.00,9.00,9.00
1,I0,0,LHRDEN,BA,0.32,0.21,9.00,9.00,9.00,9.00
2,I1,1,LHRDEN,AA,0.27,0.18,9.00,9.00,9.00,9.00
3,I1,1,LHRDEN,BA,0.27,0.18,9.00,9.00,9.00,9.00
4,I10,10,LHRDEN,AA,0.05,0.03,0.07,0.05,0.28,0.19
5,I10,10,LHRDEN,BA,0.05,0.03,0.07,0.05,0.28,0.19
6,I11,11,LHRDEN,AA,0.04,0.03,0.05,0.04,0.25,0.16
7,I11,11,LHRDEN,BA,0.04,0.03,0.05,0.04,0.25,0.16
8,I12,12,LHRDEN,AA,0.04,0.02,0.04,0.02,0.21,0.14
9,I12,12,LHRDEN,BA,0.04,0.02,0.04,0.02,0.21,0.14
